In [1]:
import os 
import json
import time
import numpy as np
import pandas as pd
from scipy.spatial import distance_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cbook as cbook
#from scipy.misc import imread
from matplotlib.pyplot import imread

from importlib import reload
%load_ext autoreload
%autoreload 2
import AnalyzeAnnotations as aa
import PlotAnnotations as pa

from tqdm import tqdm_notebook

pd.options.display.max_colwidth = 400
sns.set()

### <span style="color:red"> Load sloth jsons </span> into array of pandas dataframes

In [2]:
# load all json files from datapath into array of pandas dataframes
path ='./data/20191031_MIxedSpecies_Photos_final_annotations/'
path = './data/testdata/'

df_stats, all_df, json_files = aa.loadAllJSONSFromPath(path)
print(f"{time.strftime('%d/%m/%Y, %H:%M:%S', time.localtime())}:      Path to be loaded: {path}")
print('Number of json files loaded: ' + str(len(all_df)))

12/12/2019, 20:45:55:      Path to be loaded: ./data/testdata/
Number of json files loaded: 37


#### Calculate abundance of classes

In [3]:
# update df_stats with abundance of classes
df_stats = aa.getNumberOfClassesInDFs(df_stats, all_df, json_files)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_stats)

                                 #fish  #fish2  #fish3  #allfishes
IMG_5529_annotations_al.json       136       0       0         136
IMG_5587_annotations_al.json         9       6       0          15
IMG_5590_annotations_al.json        13       9       0          22
IMG_5616_annotations_al_jl.json    141      15       1         157
IMG_5643_annotations_al_jl.json    157      19       3         179
IMG_5666_annotations_al_jl.json    166       0       0         166
IMG_5673_annotations_al_jl.json    137       0       0         137
IMG_5681_annotations_al_jl.json    261       0       0         261
IMG_5688_annotations_al_jl.json     67       0       1          68
IMG_5693_annotations_al_jl.json     78       0       0          78
IMG_5699_annotations_al_jl.json     42       1       0          43
IMG_5701_annotations_al_jl.json    150       1       0         151
IMG_5712_annotations_al_jl.json    169       0       0         169
IMG_5718_annotations_al_jl.json    272       0       1        

#### Stats of dataframe df_stats

In [ ]:
#df.describe()
df_stats.describe()

#### Split positions + Average pos column + length column + polarization

In [12]:
df_allsplit = aa.splitPos(df_stats, all_df, json_files)

print(df_allsplit[0].head(5))

  class    xhead   xtail   yhead   ytail     x_av     y_av         len  \
0  fish   465.30  166.38  314.43  265.08  315.840  289.755  302.966316   
1  fish   813.57  547.08  197.40  253.80  680.325  225.600  272.392878   
2  fish   902.40  638.73  372.24  438.51  770.565  405.375  271.870524   
3  fish  1096.98  902.40   84.60    7.05  999.690   45.825  209.464505   
4  fish   710.64  511.83  558.36  520.29  611.235  539.325  202.422185   

    pol_rad  
0  0.163618  
1 -0.208563  
2 -0.246237  
3  0.379256  
4  0.189199  


### Plot positions

In [ ]:
#test one image
index=4
#print(json_files)

imgpath, number = pa.get_imgpath_for_json(json_files[index])
pa.plot_pos_cat_img(df_allsplit[index],imgpath, number, show=True, save=True)
pa.plot_pos_ori_cat(df_allsplit[index],imgpath, number, show=True, save=True)
pa.plot_pos_ori_cat_img(df_allsplit[index],imgpath, number, show=True, save=True)

In [ ]:
# Plot positions of all read files
pa.plot(df_allsplit, json_files, SHOW=False, SAVE=True)

print(f"{time.strftime('%d/%m/%Y, %H:%M:%S', time.localtime())}")

### Calculate distance matrix and find neighbors
* finds neighbors using a distance matrix and filtering out neighbours over two times average length and four times average length

In [ ]:
# test for one dataset
index = 4
imgpath, number = aa.get_imgpath_for_json(json_files[index])

aa.calc_neighbors(df_allsplit[index], number)
df_allsplit[index].head(2)

In [37]:
#calculate neighbors for all datasets
df_allsplit = aa.calc_all_neighbors(df_allsplit, json_files)
#show sample
df_allsplit[4].head(2)

5529:   Distance1 (av2): 469.03455 , Distance2 (av4): 938.06910 
5587:   Distance1 (av2): 376.00551 , Distance2 (av4): 752.01103 
5590:   Distance1 (av2): 400.39872 , Distance2 (av4): 800.79743 
5616:   Distance1 (av2): 275.37590 , Distance2 (av4): 550.75180 
5643:   Distance1 (av2): 259.67581 , Distance2 (av4): 519.35163 
5666:   Distance1 (av2): 409.56417 , Distance2 (av4): 819.12833 
5673:   Distance1 (av2): 497.67712 , Distance2 (av4): 995.35425 
5681:   Distance1 (av2): 430.66944 , Distance2 (av4): 861.33888 
5688:   Distance1 (av2): 473.61567 , Distance2 (av4): 947.23133 
5693:   Distance1 (av2): 425.80586 , Distance2 (av4): 851.61172 
5699:   Distance1 (av2): 421.57155 , Distance2 (av4): 843.14309 
5701:   Distance1 (av2): 427.88958 , Distance2 (av4): 855.77916 
5712:   Distance1 (av2): 419.00172 , Distance2 (av4): 838.00343 
5718:   Distance1 (av2): 388.25205 , Distance2 (av4): 776.50411 
5721:   Distance1 (av2): 431.39630 , Distance2 (av4): 862.79260 
5728:   Distance1 (av2): 

,class,xhead,xtail,yhead,ytail,x_av,y_av,len,pol_rad,nb_av2,nb_av4
0,fish,894.711757,892.919945,701.793268,838.568296,893.815851,770.180782,136.786765,-1.557697,"[1, 41, 42, 45, 46, 47]","[1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 21, 39, 40, 41, 42, 45, 46, 47, 48, 49, 54, 55, 56]"
1,fish,742.407686,820.650170,738.824061,649.830702,781.528928,694.327382,118.497697,2.291997,"[0, 2, 5, 6, 45]","[0, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 40, 41, 42, 45, 46, 47, 54, 55, 56]"


### Add number of class neighbors column
* adds columns for each fish containing the number of neighbors of each class (for distance d2/av2 and d4/av4)

In [ ]:
# test for one dataset
index = 4

aa.calc_class_neighbors(df_allsplit[index])
df_allsplit[index].head(1)

* #### for all

In [38]:
# calc class neighbors for all datasets
df_allsplit = aa.all_calc_class_neighbours(df_allsplit)
# check sample
df_allsplit[2].head(2)

,class,xhead,xtail,yhead,ytail,x_av,y_av,len,pol_rad,nb_av2,nb_av4,#neighbors_av2_f,#neighbors_av2_f2,#neighbors_av2_f3,#neighbors_av4_f,#neighbors_av4_f2,#neighbors_av4_f3
0,fish_2,3500.0,3536.0,347.0,546.0,3518.0,446.5,202.230067,-1.749765,[],[3],0,0,0,1,0,0
1,fish_2,2468.0,2482.0,441.0,603.0,2475.0,522.0,162.603813,-1.657002,[2],"[2, 3, 15, 16]",0,1,0,1,3,0


### Add average length and polarization to stats

In [42]:
all_avg_len = []
all_avg_pol = []
for idf, df_split in enumerate(tqdm_notebook(df_allsplit)):
    all_avg_len.append(df_split['len'].mean())
    
    msin = np.mean(np.sin(df_split['pol_rad']))
    mcos = np.mean(np.cos(df_split['pol_rad']))
    avg_pol = np.arctan2(msin,mcos)
    all_avg_pol.append(avg_pol)
assert(len(all_avg_len) == df_stats.shape[0])
df_stats["avg_len"] = all_avg_len
df_stats["avg_pol"] = all_avg_pol

df_stats.head(6)

,#fish,#fish2,#fish3,#allfishes,avg_len,avg_pol
IMG_5529_annotations_al.json,136,0,0,136,234.517274,-0.041841
IMG_5587_annotations_al.json,9,6,0,15,188.002757,-1.848003
IMG_5590_annotations_al.json,13,9,0,22,200.199358,-1.641929
IMG_5616_annotations_al_jl.json,141,15,1,157,137.687951,-2.017888
IMG_5643_annotations_al_jl.json,157,19,3,179,129.837906,-0.520685
IMG_5666_annotations_al_jl.json,166,0,0,166,204.782083,0.357343


### Add same class neighbors percentage and average number of neighbors (per class and total) to stats 

In [ ]:
df_stats = aa.calc_same_class_nb_p_and_avg_number_nb(df_allsplit, df_stats)
df_stats.head(5)

#### Add author to stats

#### Export to csv

In [ ]:
for i, df_split in enumerate(df_allsplit):
    number = data_numbers[i]
    df_split.to_csv('./output/stats/' + str(number) + '.csv')

df_stats.to_csv('./output/summary.csv')

## TODO

* rename columns
* refactor much
* convert .ipynb to .py
* create exe file from python script

In [36]:
angles = [0.163618, -0.208563, -0.246237, 0.379256, 0.189199]
angles_deg = np.degrees(angles)

print(angles_deg)

msin = np.mean(np.sin(angles))
mcos = np.mean(np.cos(angles))
np.degrees(np.arctan2(msin,mcos))




[  9.37462085 -11.94977966 -14.10834086  21.72976816  10.84030419]


3.1935405658203235